NOT supposed to be run as standalone but to initialize targets from other notebooks. `lvl` and `BUILDPLATFORM` can be set by calling notebook.

In [ ]:
import serial
import subprocess as sp
import sys

new_path = '../scripts/'

if new_path not in sys.path:
    sys.path.append(new_path)
import target_com as com

"""
as levels "l00", "l01", "l1" and "l3" are available.
Compare ../benchmarks/ to check which levels make sense for which board
as platforms 'cw308t-stm32f3' and 'stm32f4discovery' are available
"""
try:
	BUILDPLATFORM
except NameError:
	BUILDPLATFORM = 'cw308t-stm32f3'

try:
	lvl
except NameError:
	lvl="l00"

if "disco" in BUILDPLATFORM:
	target = serial.Serial("/dev/ttyUSB0", 38400)
elif "cw308" in BUILDPLATFORM or "415rgt" in BUILDPLATFORM:
	SCOPETYPE = 'OPENADC'
	PLATFORM = 'CWLITEARM'
	SS_VER = 'SS_VER_1_1'
	%run "Setup_Generic.ipynb"
elif "mps2" in BUILDPLATFORM:
    target = None
    pass

t_com = com.Communication_Target(target, lvl)


def reboot_flush():   
    if "cw308t" in BUILDPLATFORM:
        scope.io.nrst = False
        time.sleep(0.05)
        scope.io.nrst = "high_z"
        time.sleep(0.01)
        #Flush garbage too
        target.flush()
        time.sleep(0.5)
    elif "disco" in BUILDPLATFORM:
        target.reset_output_buffer()
        target.reset_input_buffer()
        sp.call("st-flash reset", shell=True)
        target.reset_output_buffer()
        target.reset_input_buffer()